In [1]:
import pandas as pd 
import os
import requests as rq
from bs4 import BeautifulSoup as bs
import re
import textwrap
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/114.0.0.0 Safari/537.36"
}

In [3]:
with open("APPI OPENAI.txt") as archivo:
  apikey_openai = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey_openai

In [4]:
distritos_lima = [
    "Ancón",
    "Ate",
    "Barranco",
    "Breña",
    "Carabayllo",
    "Chaclacayo",
    "Chorrillos",
    "Cieneguilla",
    "Comas",
    "El Agustino",
    "Independencia",
    "Jesús María",
    "La Molina",
    "La Victoria",
    "Lince",
    "Los Olivos",
    "Lurigancho",
    "Lurín",
    "Magdalena del Mar",
    "Miraflores",
    "Pachacámac",
    "Pucusana",
    "Pueblo Libre",
    "Puente Piedra",
    "Punta Hermosa",
    "Punta Negra",
    "Rímac",
    "San Bartolo",
    "San Borja",
    "San Isidro",
    "San Juan de Lurigancho",
    "San Juan de Miraflores",
    "San Luis",
    "San Martín de Porres",
    "San Miguel",
    "Santa Anita",
    "Santa María del Mar",
    "Santa Rosa",
    "Santiago de Surco",
    "Surquillo",
    "Villa El Salvador",
    "Villa María del Triunfo"
]


In [5]:
departamentos_peru = [
    "Amazonas",
    "Áncash",
    "Apurímac",
    "Arequipa",
    "Ayacucho",
    "Cajamarca",
    "Callao", 
    "Cusco",
    "Huancavelica",
    "Huánuco",
    "Ica",
    "Junín",
    "La Libertad",
    "Lambayeque",
    "Lima",
    "Loreto",
    "Madre de Dios",
    "Moquegua",
    "Pasco",
    "Piura",
    "Puno",
    "San Martín",
    "Tacna",
    "Tumbes",
    "Ucayali"
]


In [6]:
prompt_pais = ChatPromptTemplate.from_messages([
    ("system", """Tu tarea es identificar el o los países mencionados en una noticia.
Responde únicamente con una lista de nombres de países:
- Si se menciona un solo país, devuélvelo en una lista con un solo elemento.
- Si se mencionan varios países, devuélvelos todos en una lista.
- Si no se menciona ningún país, devuelve una lista vacía.
No des explicaciones ni añadas texto adicional. Solo responde con la lista de países."""),
    ("human", "{input}")
])

In [7]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4")

def pais_detector(texto: str) -> list:
    respuesta = llm.invoke(prompt_pais.format_messages(input=texto)).content.strip().lower()
    return respuesta

def buscar_ordenados(texto, lista):
    texto_lower = texto.lower()
    encontrados = []
    for lugar in lista:
        patron = r'\b' + re.escape(lugar.lower()) + r'\b'
        match = re.search(patron, texto_lower)
        if match:
            encontrados.append((lugar, match.start()))  
    encontrados.sort(key=lambda x: x[1])
    return [nombre for nombre, _ in encontrados]

In [8]:
lista_url = []
lista_titulo = []
lista_texto = []
lista_fecha = []
lista_distrito = []
lista_departamento = []
lista_pais = []

In [9]:
for pagina in tqdm(range(1,20)):
    url = f'https://elcomercio.pe/buscar/feminicidio/todas/descendiente/{pagina}/'
    try: 
        html = rq.get(url,headers=headers).content.decode("utf-8")
        soup = bs(html) #creamos nuestro arbol
        enlaces_raw = soup.find_all('div',class_="paginated-list paginated-list--search")[0].find_all('a',class_="story-item__link h-full")
        links = ['https://elcomercio.pe'+ link.get('href') for link in enlaces_raw if 'href' in link.attrs ]
        for noticia in tqdm(links): 
            try:
                soup2 = bs(rq.get(noticia, headers=headers).content.decode("utf-8")) 
                try:
                    titulo = soup2.find_all("h1")[0].text
                    fecha = "/".join(list((re.findall(r"(\d{2})[/-](\d{2})[/-](\d{4})",soup2.find_all('time')[0].text))[0]))
                    body = " ".join([p.text for p in soup2.find_all('div', class_='story-contents w-full')[0].find_all('p') 
                    if 'dir' not in p.attrs and 'lang' not in p.attrs and p.get('class') != ['story-contents__about-author-bio']])
                    body2 = titulo  + ' ' + body
                    distrito = buscar_ordenados(body2, distritos_lima)
                    departamento = buscar_ordenados(body2, departamentos_peru)
                    pais = pais_detector(body)
                    lista_pais.append(pais)
                    lista_url.append(noticia)
                    lista_titulo.append(titulo)
                    lista_texto.append(body)
                    lista_fecha.append(fecha)
                    lista_departamento.append(departamento)
                    lista_distrito.append(distrito)
                except:
                    print(f'Error al encontrar metadata de la noticia: No se encontro metadata: {noticia}\n')
            except Exception as e:
                print(f"Error al procesar noticia: {noticia}\n{e}")
    except Exception as e:
        print(f"Error al cargar la página {pagina}:\n{e}")

  0%|          | 0/19 [00:00<?, ?it/s]

Error al encontrar metadata de la noticia: No se encontro metadata: https://elcomercio.pe/mag/usa/local-us/feminicidio-en-brooklyn-38-anos-de-carcel-para-el-asesino-de-sugerys-ramirez-nnda-nnlt-noticia/



Error al encontrar metadata de la noticia: No se encontro metadata: https://elcomercio.pe/lima/policiales/feminicidio-en-comas-el-caso-que-involucra-a-oficial-pnp-con-denuncias-sheyla-darwin-condori-antenaza-ministerio-del-interior-policia-nacional-tenemos-que-hablar-noticia/



  5%|▌         | 1/19 [01:25<25:44, 85.83s/it]

Error al encontrar metadata de la noticia: No se encontro metadata: https://elcomercio.pe/mag/respuestas/us/que-significa-un-feminicidio-nnda-nnlt-noticia/



100%|██████████| 13/13 [00:30<00:00,  2.37s/it]
0it [00:00, ?it/s]/19 [01:58<15:30, 54.76s/it]
0it [00:00, ?it/s]/19 [02:00<08:10, 30.65s/it]
0it [00:00, ?it/s]/19 [02:03<04:51, 19.44s/it]
0it [00:00, ?it/s]/19 [02:05<03:05, 13.25s/it]
0it [00:00, ?it/s]/19 [02:07<02:03,  9.50s/it]
0it [00:00, ?it/s]/19 [02:09<01:24,  7.03s/it]
0it [00:00, ?it/s]/19 [02:11<01:00,  5.53s/it]
0it [00:00, ?it/s]/19 [02:13<00:44,  4.43s/it]
0it [00:00, ?it/s]0/19 [02:16<00:33,  3.75s/it]
0it [00:00, ?it/s]1/19 [02:18<00:25,  3.24s/it]
0it [00:00, ?it/s]2/19 [02:20<00:19,  2.84s/it]
0it [00:00, ?it/s]3/19 [02:22<00:15,  2.60s/it]
0it [00:00, ?it/s]4/19 [02:23<00:11,  2.33s/it]
0it [00:00, ?it/s]5/19 [02:25<00:08,  2.25s/it]
0it [00:00, ?it/s]6/19 [02:29<00:07,  2.62s/it]
0it [00:00, ?it/s]7/19 [02:31<00:05,  2.54s/it]
0it [00:00, ?it/s]8/19 [02:34<00:02,  2.52s/it]
100%|██████████| 19/19 [02:36<00:00,  8.22s/it]


In [10]:
df = pd.DataFrame({
    'url': lista_url,
    'titulo': lista_titulo,
    'texto': lista_texto,
    'fecha': lista_fecha,
    'distrito': lista_distrito,
    'departamento': lista_departamento,
    'pais': lista_pais})

In [11]:
df

,url,titulo,texto,fecha,distrito,departamento,pais
0,https://elcomercio.pe/lima/policiales/feminici...,Feminicidio en San Isidro: sujeto planificó a ...,Juan Carlos Olórtegui Dueñas (28) lo tenía to...,08/04/2025,"[San Isidro, Miraflores, Lince]","[Arequipa, Lima, Junín]","[""perú""]"
1,https://elcomercio.pe/lima/sucesos/feminicidio...,Feminicidio en San Isidro: sujeto asesina a ba...,El 30 de marzo se reportó un feminicidio en Ar...,03/04/2025,[San Isidro],"[Arequipa, Lima, Áncash, Cusco, Ica, Tacna]","[""perú""]"
2,https://elcomercio.pe/mundo/europa/dia-interna...,Miles de mujeres marchan en Italia contra el m...,Miles de personas se manifestaron este sábado ...,08/03/2025,[],[La Libertad],"[""italia"", ""palestina"", ""japón""]"
3,https://elcomercio.pe/lima/jean-deza-tras-audi...,Jean Deza tras audiencia por tentativa de femi...,Jean Deza reapareció tras la audiencia de pris...,05/02/2025,[],[],[]
4,https://elcomercio.pe/lima/judiciales/jean-dez...,Jean Deza: PJ reprograma audiencia de prisión ...,El Poder Judicial reprogramó la audiencia de p...,23/01/2025,[San Miguel],"[Lima, Lambayeque]","[""perú""]"
5,https://elcomercio.pe/lima/sucesos/jean-deza-g...,Jean Deza: solicitan nueve meses de prisión pr...,La Segunda Fiscalía Especializada en Violencia...,17/01/2025,[San Miguel],[Lima],"[""perú""]"
6,https://elcomercio.pe/opinion/columnistas/copa...,"Copa Juvenil Mitsubishi Asia 2025, el retraso ...",Este jueves en el Semáforo conoce la postura d...,17/01/2025,[El Agustino],[],[]
7,https://elcomercio.pe/lima/sucesos/jean-deza-p...,Jean Deza pasó por una evaluación psiquiátrica...,El futbolista Jean Deza reapareció y se pronun...,11/01/2025,"[San Miguel, La Victoria]",[],[]
8,https://elcomercio.pe/lima/policiales/sheyla-c...,Sheyla Cóndor: ¿Dónde están y qué se sabe de l...,"El 13 de noviembre, la familia de Sheyla Cóndo...",09/01/2025,"[San Juan de Lurigancho, Lurigancho, Comas, In...",[Lima],"[""perú"", ""chile""]"
9,https://elcomercio.pe/lima/sucesos/san-martin-...,Intento de feminicidio en SMP: hombre apuñala ...,"En el asentamiento humano Húsares de Junín, en...",08/01/2025,[San Martín de Porres],"[Junín, San Martín]","[""perú""]"


In [12]:
df.to_excel('Noticias_ElComercio.xlsx',index=False)